In [ ]:
#랜덤포레스트

In [4]:
def get_new_feature_name_df(old):
    import pandas as pd
    dup = pd.DataFrame(data=old.groupby('column_name').cumcount(), columns=['dup_cnt'])
    dup = dup.reset_index()
    # print(dup.head())
    # print(old.reset_index().head())
    new = pd.merge(old.reset_index(), dup, how='outer')
    new['column_name'] = new[['column_name', 'dup_cnt']].apply(lambda x:x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)
    new.drop(columns=['index'], inplace=True)
    return new

In [5]:
def get_human_dataset():
    import pandas as pd
    feature_name_df = pd.read_csv('./human_activity/features.txt', sep = '\s+',
                                header=None, names=['column_index', 'column_name'])
    new_name = get_new_feature_name_df(feature_name_df)
    feature_name = new_name.iloc[:,1].values.tolist()
    X_train = pd.read_csv('./human_activity/train/X_train.txt', sep = '\s+', names =feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt', sep = '\s+', names =feature_name)
    y_train = pd.read_csv('./human_activity/train/y_train.txt', sep = '\s+', names =['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt', sep = '\s+', names =['action'])
    return X_train, X_test, y_train, y_test

In [6]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings ('ignore')

In [15]:
rf_clf = RandomForestClassifier(random_state=0, max_depth=8)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9195792331184255

In [10]:
from sklearn.model_selection import GridSearchCV

In [14]:
%%time
param = {
    'max_depth':[8, 16, 24],
    'min_samples_split': [1, 6, 12],
    'min_samples_leaf': [2, 8, 16]
}
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=param, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'max_depth': 16, 'min_samples_leaf': 8, 'min_samples_split': 6}
0.9178454842219804
CPU times: total: 12 s
Wall time: 13.5 s


In [18]:
pred = grid_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.9222938581608415

In [22]:
rf_clf = RandomForestClassifier(n_estimators=500, 
                                n_jobs =-1,
                                max_depth=16, 
                                min_samples_leaf = 8,
                                min_samples_split=6, 
                                random_state=0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9192399049881235

In [23]:
#GBM
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
%%time
#X_train, X_test, y_train, y_test = get_human_dataset()
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
pred = gb_clf.predict(X_test)
accuracy_score(y_test, pred)

CPU times: total: 11min 1s
Wall time: 11min 1s


0.9389209365456397

In [ ]:
#XGBoost > 매우 뛰어난 부스팅 알고리즘 ,, 시간 많이 걸림ㅠ 

In [ ]:
#LightGBM > XG보다 빠르고 메모리 사용량도 적음